# LSTM을 이용한 텍스트 생성
- 데이터 다운로드: 캐글 [New York Times Comments](https://www.kaggle.com/aashita/nyt-comments)

In [ ]:
## https://wikidocs.net/45101

## 1. 데이터 전처리

In [31]:
import numpy as np 
import pandas as pd
from string import punctuation

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [32]:
df=pd.read_csv('data/ArticlesMay2017.csv', encoding = 'latin')
df.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,5906b3197c459f24986dd003,2412,By IAN JOHNSON,article,My Beijing: The Sacred City,"['Travel and Vacations', 'Beijing (China)', 'R...",3,Travel,1,2017-05-01 04:01:21,Unknown,"This metropolis was once a total work of art, ...",The New York Times,News,https://www.nytimes.com/2017/05/01/travel/beij...
1,NaN,5906b3297c459f24986dd004,2318,By EMMA G. FITZSIMMONS,article,"6 Million Riders a Day, 1930s Technology","['Subways', 'Delays (Transportation)', 'Transi...",68,Metro,1,2017-05-01 04:01:33,Unknown,New Yorkâs subway is struggling with old inf...,The New York Times,News,https://www.nytimes.com/2017/05/01/nyregion/ne...
2,NaN,5906ceec7c459f24986dd021,1795,By MARC TRACY,article,Seeking a Cross-Border Conference,"['Cetys University', 'College Athletics', 'Nat...",3,Sports,1,2017-05-01 06:00:05,Unknown,Cetys University is making an ambitious bid to...,The New York Times,News,https://www.nytimes.com/2017/05/01/sports/mexi...
3,NaN,5906cfa77c459f24986dd022,213,By SHANNON DOYNE,article,"Questions for: âDespite the âYuck Factor,â...",[],3,Learning,0,2017-05-01 06:03:03,Unknown,How are leeches used to treat various medical ...,The New York Times,News,https://www.nytimes.com/2017/05/01/learning/qu...
4,NaN,5906e1c07c459f24986dd039,1342,By JASON STANLEY,article,Who Is a âCriminalâ?,"['Illegal Immigration', 'Traffic and Parking V...",3,OpEd,0,2017-05-01 07:20:26,Unknown,Justice Roberts was right. The Trump administr...,The New York Times,Op-Ed,https://www.nytimes.com/2017/05/01/opinion/who...


In [33]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  16
Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [34]:
# headline 열의 데이터만 사용, Null 검사
df['headline'].isnull().values.any()

False

In [35]:
# headline 열의 데이터로 리스트 생성
headline = [title for title in df.headline.values]
headline[:5]

['My Beijing: The Sacred City',
 '6 Million Riders a Day, 1930s Technology',
 'Seeking a Cross-Border Conference',
 'Questions for: â\x80\x98Despite the â\x80\x9cYuck Factor,â\x80\x9d Leeches Are Big in Russian Medicineâ\x80\x99',
 'Who Is a â\x80\x98Criminalâ\x80\x99?']

In [36]:
len(headline)

996

In [37]:
# 노이즈 데이터('Unknown') 제거
headline = [title for title in headline if title != 'Unknown']
len(headline)

935

In [38]:
# 구두점 제거와 소문자화를 위한 함수
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() 

In [39]:
text = [repreprocessing(x) for x in headline]
text[:5]

['my beijing the sacred city',
 '6 million riders a day 1930s technology',
 'seeking a crossborder conference',
 'questions for despite the yuck factor leeches are big in russian medicine',
 'who is a criminal']

In [40]:
# 단어 집합(vocabulary)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 2653


In [41]:
print(t.word_index)

{'the': 1, 'a': 2, 'to': 3, 'of': 4, 'in': 5, 'for': 6, 'and': 7, 'trump': 8, 'is': 9, 'on': 10, 'new': 11, 'how': 12, 'with': 13, 'you': 14, 'at': 15, 'an': 16, 'season': 17, 'what': 18, 'it': 19, 'its': 20, 'york': 21, 'episode': 22, 'trumps': 23, 'from': 24, 'as': 25, 'be': 26, 'not': 27, 'are': 28, 'may': 29, 'when': 30, 'but': 31, 'after': 32, 'about': 33, '3': 34, 'today': 35, 'this': 36, 'us': 37, 'recap': 38, 'by': 39, 'we': 40, 'comey': 41, 'house': 42, 'questions': 43, 'health': 44, 'president': 45, 'bill': 46, 'that': 47, 'my': 48, 'too': 49, 'get': 50, 'was': 51, 'life': 52, 'or': 53, 'will': 54, 'why': 55, 'your': 56, 'city': 57, 'dont': 58, 'care': 59, 'more': 60, 'do': 61, 'can': 62, 'has': 63, 'no': 64, 'up': 65, 'time': 66, 'out': 67, 'into': 68, 'better': 69, 'war': 70, 'his': 71, 'their': 72, 'says': 73, 'american': 74, 'love': 75, 'one': 76, 'gop': 77, '5': 78, 'climate': 79, 'should': 80, 'donald': 81, 'getting': 82, 'mr': 83, 'james': 84, 'like': 85, 'first': 86, 

In [78]:
sequences = []
for line in text: # 996 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[48, 799],
 [48, 799, 1],
 [48, 799, 1, 800],
 [48, 799, 1, 800, 57],
 [114, 406],
 [114, 406, 407],
 [114, 406, 407, 2],
 [114, 406, 407, 2, 179],
 [114, 406, 407, 2, 179, 801],
 [114, 406, 407, 2, 179, 801, 802],
 [803, 2]]

###### 실제문장 -> 확인

In [67]:
sequences.shape

(5501, 16)

In [106]:
t.word_index['a']

2

In [79]:
## a로 시작하는 문장 찾기
start_a = []
for i in range(len(sequences)):
    if (sequences[i][0] == 2) :
        start_a.append(sequences[i])
start_a

[[2, 841],
 [2, 841, 430],
 [2, 882],
 [2, 882, 883],
 [2, 882, 883, 194],
 [2, 882, 883, 194, 6],
 [2, 882, 883, 194, 6, 1],
 [2, 882, 883, 194, 6, 1, 11],
 [2, 882, 883, 194, 6, 1, 11, 884],
 [2, 882, 883, 194, 6, 1, 11, 884, 456],
 [2, 882, 883, 194, 6, 1, 11, 884, 456, 5],
 [2, 882, 883, 194, 6, 1, 11, 884, 456, 5, 48],
 [2, 882, 883, 194, 6, 1, 11, 884, 456, 5, 48, 52],
 [2, 904],
 [2, 904, 465],
 [2, 904, 465, 905],
 [2, 906],
 [2, 906, 466],
 [2, 906, 466, 4],
 [2, 906, 466, 4, 71],
 [2, 906, 466, 4, 71, 120],
 [2, 471],
 [2, 471, 6],
 [2, 471, 6, 472],
 [2, 471, 6, 472, 473],
 [2, 471, 6, 472, 473, 13],
 [2, 471, 6, 472, 473, 13, 920],
 [2, 154],
 [2, 154, 283],
 [2, 154, 283, 3],
 [2, 154, 283, 3, 475],
 [2, 927],
 [2, 927, 928],
 [2, 927, 928, 284],
 [2, 927, 928, 284, 1],
 [2, 927, 928, 284, 1, 929],
 [2, 927, 928, 284, 1, 929, 4],
 [2, 927, 928, 284, 1, 929, 4, 930],
 [2, 499],
 [2, 499, 974],
 [2, 499, 974, 31],
 [2, 499, 974, 31, 27],
 [2, 499, 974, 31, 27, 2],
 [2, 499, 

In [92]:
start_a_1 = []
for i in range(len(start_a)):
    start_a_1.append(t.sequences_to_texts([start_a[i]]))
    
start_a_1

[['a bettertasting'],
 ['a bettertasting bird'],
 ['a fantastic'],
 ['a fantastic fantastical'],
 ['a fantastic fantastical book'],
 ['a fantastic fantastical book for'],
 ['a fantastic fantastical book for the'],
 ['a fantastic fantastical book for the new'],
 ['a fantastic fantastical book for the new cool'],
 ['a fantastic fantastical book for the new cool woman'],
 ['a fantastic fantastical book for the new cool woman in'],
 ['a fantastic fantastical book for the new cool woman in my'],
 ['a fantastic fantastical book for the new cool woman in my life'],
 ['a oneday'],
 ['a oneday crossword'],
 ['a oneday crossword internship'],
 ['a vegetable'],
 ['a vegetable patch'],
 ['a vegetable patch of'],
 ['a vegetable patch of his'],
 ['a vegetable patch of his own'],
 ['a search'],
 ['a search for'],
 ['a search for dad'],
 ['a search for dad along'],
 ['a search for dad along with'],
 ['a search for dad along with nessie'],
 ['a high'],
 ['a high bar'],
 ['a high bar to'],
 ['a high bar

In [43]:
index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 1번 단어 :', index_to_word[1])
print('빈도수 상위 582번 단어 :', index_to_word[582]) 

빈도수 상위 1번 단어 : the
빈도수 상위 582번 단어 : taiwan


In [94]:
max_len=max(len(s) for s in sequences)
print('샘플의 최대 길이 :', max_len) 

샘플의 최대 길이 : 16


In [95]:
# 전체 샘플의 길이를 24(가장 긴 샘플의 길이)로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0  48 799]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  48 799   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0  48 799   1 800]]


In [46]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [47]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)

In [48]:
X.shape, y.shape

((5501, 15), (5501, 2653))

## 2. 모델 설계 및 학습

In [49]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

In [50]:
# 임베딩 벡터는 10차원, 은닉 상태 크기는 128
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1, name="Embedding_Layer"))
model.add( LSTM(128, name="LSTM_Layer"))
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 15, 10)            26530     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 2653)              342237    
Total params: 439,935
Trainable params: 439,935
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [52]:
history = model.fit(X, y, epochs=200, verbose=2) 

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
 - 2s - loss: 7.4989 - accuracy: 0.0349
Epoch 2/200
 - 2s - loss: 6.9931 - accuracy: 0.0369
Epoch 3/200
 - 2s - loss: 6.8696 - accuracy: 0.0365
Epoch 4/200
 - 2s - loss: 6.7790 - accuracy: 0.0380
Epoch 5/200
 - 2s - loss: 6.6745 - accuracy: 0.0471
Epoch 6/200
 - 2s - loss: 6.5428 - accuracy: 0.0489
Epoch 7/200
 - 2s - loss: 6.3875 - accuracy: 0.0527
Epoch 8/200
 - 2s - loss: 6.2123 - accuracy: 0.0545
Epoch 9/200
 - 2s - loss: 6.0362 - accuracy: 0.0580
Epoch 10/200
 - 2s - loss: 5.8610 - accuracy: 0.0678
Epoch 11/200
 - 2s - loss: 5.6911 - accuracy: 0.0762
Epoch 12/200
 - 2s - loss: 5.5331 - accuracy: 0.0763
Epoch 13/200
 - 2s - loss: 5.3814 - accuracy: 0.0825
Epoch 14/200
 - 2s - loss: 5.2368 - accuracy: 0.0869
Epoch 15/200
 - 2s - loss: 5.0985 - accuracy: 0.0896
Epoch 16/200
 - 2s - loss: 4.9633 - accuracy: 0.0978
Epoch 17/200
 - 2s - loss: 4.8351 - accuracy: 0.1027
Epoch 18/200
 - 2s - loss: 4.7108 - accuracy: 0.1122
Epoch 19/200
 - 2s - loss: 4.5893 - accuracy: 0.1240
Ep

Epoch 155/200
 - 2s - loss: 0.4238 - accuracy: 0.8917
Epoch 156/200
 - 2s - loss: 0.4196 - accuracy: 0.8937
Epoch 157/200
 - 2s - loss: 0.4169 - accuracy: 0.8935
Epoch 158/200
 - 2s - loss: 0.4115 - accuracy: 0.8937
Epoch 159/200
 - 2s - loss: 0.4097 - accuracy: 0.8927
Epoch 160/200
 - 2s - loss: 0.4055 - accuracy: 0.8929
Epoch 161/200
 - 2s - loss: 0.4015 - accuracy: 0.8949
Epoch 162/200
 - 2s - loss: 0.3973 - accuracy: 0.8920
Epoch 163/200
 - 2s - loss: 0.3956 - accuracy: 0.8949
Epoch 164/200
 - 2s - loss: 0.3916 - accuracy: 0.8946
Epoch 165/200
 - 2s - loss: 0.3898 - accuracy: 0.8955
Epoch 166/200
 - 2s - loss: 0.3866 - accuracy: 0.8947
Epoch 167/200
 - 2s - loss: 0.3836 - accuracy: 0.8960
Epoch 168/200
 - 2s - loss: 0.3808 - accuracy: 0.8935
Epoch 169/200
 - 2s - loss: 0.3799 - accuracy: 0.8938
Epoch 170/200
 - 2s - loss: 0.4218 - accuracy: 0.8862
Epoch 171/200
 - 2s - loss: 0.4071 - accuracy: 0.8900
Epoch 172/200
 - 2s - loss: 0.3814 - accuracy: 0.8940
Epoch 173/200
 - 2s - loss: 

## 3. 모델 검증

In [62]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=15, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [63]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

i accidentally killed a child may i contact the family tracks


In [64]:
print(sentence_generation(model, t, 'how', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성

how to be mindful while taking a shower idea of a


In [66]:
print(sentence_generation(model, t, 'former', 10))

former charge first about it to learn from over had trump


###### 예측문장 ->보여주기

In [103]:
print(sentence_generation(model, t, 'a', 6))  ##예측 문장

a new entrant in the factcheck biz


In [101]:
start_a_1[71]  #실제문장

['a new entrant in the factcheck biz']